El principal objetivo de este notebook es un análisis simple de los datos abiertos de Airbnb. Este dataset contiene los datos de Airbnb en la Ciudad de México y fue tomado de [[Inside Airbnb](http://insideairbnb.com/get-the-data.html)](http://insideairbnb.com/get-the-data.html).

La fecha de compilación del dataset es del **23 de diciembre de 2020**

In [ ]:
#Librerias

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud, STOPWORDS 

In [ ]:
#Cargar el dataset
data = pd.read_csv("../input/airbnb-mexico-city-distrito-federal-mexico/listings.csv")
data.head()

# Análisis exploratorio

In [ ]:
data["room_type"].value_counts()

In [ ]:
data["neighbourhood"].value_counts()

In [ ]:
len(data["host_id"].unique())

Después de hacer el análisis exploratorio y observar los datos podemos empezar a sacar las primeras conclusiones.

* La delegación Cuauhtémoc es la que tiene más anuncios.
* El tipo de habitación que más se oferta es "Entire home/apt", casa/departamento entero.
* Hay 11,412 anfitriones.


# Proporciones

In [ ]:
host = data["host_id"].value_counts().rename_axis('Id').to_frame('Count')
data_h = data[["host_id","host_name"]]

host_merged = pd.merge(left = data_h, right = host,
                       how = "right", left_on = "host_id", right_on = "Id")
host_merged = host_merged.drop_duplicates(subset="host_id")

host_merged.head(15)

Como podemos observar en el dataframe, existen anfitriones con una gran cantidad de anuncios en la plataforma, de los cuales
la gran mayoría probablemente se trate de inmobiliarios.

> Algunos vivales del sector inmobiliario construyen edificios de departamentos y no los venden, sino que los ponen en renta con esta plataforma 
    De acuedo con lo que dijo el responsable de turismo en la Ciudad de México, **Carlos Mackinlay** en [El Universal](https://www.eluniversal.com.mx/metropoli/cdmx/urgen-regular-airbnb-y-otras-aplicaciones-de-hospedaje-en-la-cdmx)
   
El anfitrión com más anuncios en la CDMX es "HOMi", después de investigar en interntet se encontro que se trata de **un corporativo dedicado a la administración integral de inmuebles, especializados en la renta de corto plazo** que opera desde el 2016. [HOMi](https://vymaps.com/MX/HOMi-Suits-lofts-Residences-633108513767722/)

[Perfil de Airbnb](https://www.airbnb.mx/users/show/91265490)

In [ ]:
host_min = host_merged[host_merged["Count"]== 1]
host_max = len(host_merged)-len(host_min)

h1 = round((len(host_min)/len(host_merged))*100,2)
h2 = round((host_max/len(host_merged))*100,2)

anfitriones = [h1,h2]

print("Anfitriones con solo un anuncio: ", len(host_min))
print("Anfitriones con más de un anuncio: ", host_max)
print("Total de anfitriones: ", len(host_merged))

In [ ]:
labels = ["Anfitriones con solo un anuncio", "Anfitriones con más de un anuncio"]
sizes = anfitriones
colors = ['lightskyblue', 'lightcoral']
#explode = (0, 0.1, 0)
#patches, texts = plt.pie(sizes, explode=explode, colors=colors, shadow=True, startangle=90)
plt.pie(sizes, colors=colors, radius=6.5,
autopct='%1.1f%%', shadow=True, startangle=90)
plt.title("Anfitriones con más de un anuncio vs Anfitriones con solo un anuncio")
plt.legend(labels, loc="best")

plt.axis('equal')
plt.show()

Podemos observar que poco más de una cuarta parte de los anfitriones tiene más de un anuncio.

In [ ]:
%matplotlib inline
table= pd.crosstab(data.neighbourhood, data.room_type)
table.div(table.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.title("Proporción de los tipos de habitación por delegación")
plt.xlabel("Delegación")
plt.ylabel("Habitación")

Hay tres delegaciones en donde más de la mitad de los anuncios corresponden a casas/departamentos enteros.
* Cuajimalpa de Morelos 
* Cuauhtémoc
* Miguel Hidalgo

In [ ]:
d1 = data[data["room_type"]=="Private room"]
d2 = data[data["room_type"]=="Shared room"]
d3 = data[data["room_type"]=="Entire home/apt"]
d4 = data[data["room_type"]=="Hotel room"]
d1 = round((len(d1)/len(data))*100,2)
d2 = round((len(d2)/len(data))*100,2)
d3 = round((len(d3)/len(data))*100,2)
d4 = round((len(d4)/len(data))*100,2)

print("=========Porcentajes=============")
print("Entire home/apt: ", d3,"%" ,"Private room: ", d1,"%" ,"Shared room: ", d2,"%","Hotel room: ", d4,"%")
pct = [d3, d1, d2, d4]

In [ ]:
labels = ["Entire home/apt", "Private room", "Shared room", "Hotel room"]
sizes = pct
colors = ['yellowgreen', 'lightskyblue', 'lightcoral', 'gold']
#explode = (0, 0.1, 0)
#patches, texts = plt.pie(sizes, explode=explode, colors=colors, shadow=True, startangle=90)
plt.pie(sizes, colors=colors, radius=6.5,
autopct='%1.1f%%', shadow=True, startangle=90)
plt.title("Proporción del tipo de habitación")
plt.legend(labels, loc="best")

plt.axis('equal')
plt.show()

# Relación entre el precio de renta y las veces que se ha rentado

In [ ]:
# Ajustar plot y subplots 

sns.relplot(x="price", y="calculated_host_listings_count",
    data=data,
    kind="scatter",
    hue = 'room_type',
    palette= 'hls',
    col = "room_type",
    col_order=["Entire home/apt", "Private room", "Shared room", "Hotel room"])
# Mostrar plot
plt.show()

Como podemos ver en la gráfica de arriba, independientemente del tipo de habitación **entre más barato sea el inmueble mayor es la cantidad de veces que se renta**.

# Relación entre las reseñas y las veces que se ha rentado

In [ ]:
# Ajustar plot y subplots 

sns.relplot(x="number_of_reviews", y="calculated_host_listings_count",
    data=data,
    kind="scatter",
    hue = 'room_type',
    palette= 'hls',
    col = "room_type",
    col_order=["Entire home/apt", "Private room", "Shared room", "Hotel room"])
# Mostrar plot
plt.show()

De igual forma que el precio, **la cantidad de reseñas no tiene relación con la cantidad de veces que se renta .**

# Nube de términos

In [ ]:
comment_words = '' 
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for val in data["name"]: 
      
    # typecaste each val to string 
    val = str(val) 
  
    # split the value 
    tokens = val.split() 
      
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

Como podemos observar en la nube de términos de los títulos de los anuncios en la CDMX, las zonas que más se repiten en la nube corresponden a zonas de gran plusvalía como "Condesa" o "Polanco". También podemos notar que los inmuebles más anunciados son "apartment", "suite", "room", "loft", probablemente el hecho que que estén anunciados en inglés se deba a la alta demanda de extranjeros al hospedarse en las zonas anteriormente mencionadas. [CDMX, el mejor destino para los extranjeros según AirBnb](https://www.capitalmexico.com.mx/zen/cdmx-el-mejor-destino-para-los-extranjeros-segun-airbnb/)